In [14]:
import pandas as pd
import seaborn as sb
import csv
from sympy import FiniteSet

#import pysam
import os
import math
import random
import numpy as np

from pathlib import Path
from random import seed
from random import randint
from functools import reduce
#from Numeric import *
#from LinearAlgebra import singular_value_decomposition
from operator import add as concat
#from more_itertools import powerset
from itertools import combinations
from itertools import chain, combinations


In [15]:

def powerset(fullset):
  listsub = list(fullset)
  subsets = []
  for i in range(2**len(listsub)):
    subset = []
    for k in range(len(listsub)):            
      if i & 1<<k:
        subset.append(listsub[k])
    subsets.append(subset)        
  return subsets

def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

In [3]:
def Sy(dic,X):
    value = []
    for k in range(len(X)):
        value.append({i for i in dic if dic[i]==X[k]})
    return value
def lenSy(dic,X):
    value=[]
    for k in range(len(X)):
        value.append(len({i for i in dic if dic[i]==X[k]}))
    return value
def lenIntersecciones(X):
    value=[]
    for k in range(len(X)):
        value.append(len(X[k]))
    return value

In [4]:
def depth_Y2(conjunto,A):
    m=len(A)
    if os.stat(ruta_depth + conjunto + '-Simulados-subsp_capsici_1101_sorted.txt').st_size > 0:
        d1 = pd.read_csv(ruta_depth + conjunto + '-Simulados-subsp_capsici_1101_sorted.txt', sep='\t', header=None)
        a1=d1[2].sum()
        l1=len(d1)
        dyx1=a1/(l1)*1/m**2
    else:
        dyx1=0
    if os.stat(ruta_depth +conjunto + '-Simulados-subsp_nebraskensis_61-1_sorted.txt').st_size > 0:
        d2 = pd.read_csv(ruta_depth + conjunto + '-Simulados-subsp_nebraskensis_61-1_sorted.txt', sep='\t', header=None)
        a2=d2[2].sum()
        l2=len(d2)
        dyx2=a2/(l2)*1/m**2
    else:
        dyx2=0
    if os.stat(ruta_depth + conjunto + '-Simulados-subsp_michi_contigs_sorted.txt').st_size > 0:
        d3 = pd.read_csv(ruta_depth + conjunto + '-Simulados-subsp_michi_contigs_sorted.txt', sep='\t', header=None)
        a3=d3[2].sum()
        l3=len(d3)
        dyx3=a3/(l3)*1/m**2
    else:
        dyx3=0
    return dyx1,dyx2,dyx3    

In [5]:
#calculo de profundiades previa
def depth_Y(conjunto,A):
    d1 = pd.read_csv(ruta_depth + conjunto+ '-Simulados-subsp_capsici_1101_sorted.txt', sep='\t', header=None) #importar las tablas para cada genoma
    d2 = pd.read_csv(ruta_depth + conjunto+ '-Simulados-subsp_nebraskensis_61-1_sorted.txt', sep='\t', header=None)
    d3 = pd.read_csv(ruta_depth + conjunto+ '-Simulados-subsp_michi_contigs_sorted.txt', sep='\t', header=None) 
    a1=d1[2].sum()
    a2=d1[2].sum()
    a3=d1[2].sum()
    ay=(a1+a2+a3)/3
    l1=len(d1)
    l2=len(d2)
    l3=len(d3)
    m=len(A)
    dyx1=ay/(l1)*1/m**2
    dyx2=ay/(l2)*1/m**2
    dyx3=ay/(l3)*1/m**2
    return dyx1,dyx2,dyx3

In [23]:
folder1='2TP' #nombre donde esta los genomas
#folder_bash=1TP

In [21]:
%%bash -s "$folder1"
ls genomes/$1/*fna

genomes/6TP/Clavibacter_michiganensis_subsp_capsici_1101.fna
genomes/6TP/Clavibacter_michiganensis_subsp_insidiosus_ATCC_10253.fna
genomes/6TP/Clavibacter_michiganensis_subsp_michi_contigs.fna
genomes/6TP/Clavibacter_michiganensis_subsp_nebraskensis_419B.fna
genomes/6TP/Clavibacter_michiganensis_subsp_sepedonicus_ATCC33113.fna
genomes/6TP/Clavibacter_michiganensis_subsp_tessellarius_ATCC_33566.fna


In [24]:
os.makedirs('output-tda/'+folder1)
os.makedirs('output-tda/'+folder1+'/index-bowtie')
os.makedirs('output-tda/'+folder1+'/out-bowtie')
os.makedirs('output-tda/'+folder1+'/bam')
os.makedirs('output-tda/'+folder1+'/sorted')
os.makedirs('output-tda/'+folder1+'/tablas')
os.makedirs('output-tda/'+folder1+'/SyReads')
os.makedirs('output-tda/'+folder1+'/sam_extract')
os.makedirs('output-tda/'+folder1+'/depth')

In [ ]:
%%bash -s "$folder1"
source /opt/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics
ls genomes/$1/*fna | while read line; 
do 
    id=$(echo $line| rev| cut -d'_' -f1,2,3 |rev |cut -d'.' -f1);
    bowtie2-build $line output-tda/$1/index-bowtie/$id;
    echo "$id" >>list-genomasrefencia6TP.txt
done

In [25]:
%%bash -s "$folder1"
source /opt/anaconda3/etc/profile.d/conda.sh
conda activate metagenomics

cat list-genomasrefencia6TP.txt | while read line;
do
bowtie2 -x output-tda/6TP/index-bowtie/$line -1 ReadsSimulados/metagenome_simulated/metagenome_2TP_R1.fastq -2 ReadsSimulados/metagenome_simulated/metagenome_2TP_R2.fastq --no-unal -p 12 -S output-tda/$1/out-bowtie/Simulados-${line}.sam 
done

2000 reads; of these:
  1596 (79.80%) were paired; of these:
    1596 (100.00%) aligned concordantly 0 times
    0 (0.00%) aligned concordantly exactly 1 time
    0 (0.00%) aligned concordantly >1 times
    ----
    1596 pairs aligned concordantly 0 times; of these:
      930 (58.27%) aligned discordantly 1 time
    ----
    666 pairs aligned 0 times concordantly or discordantly; of these:
      1332 mates make up the pairs; of these:
        1166 (87.54%) aligned 0 times
        145 (10.89%) aligned exactly 1 time
        21 (1.58%) aligned >1 times
  404 (20.20%) were unpaired; of these:
    200 (49.50%) aligned 0 times
    204 (50.50%) aligned exactly 1 time
    0 (0.00%) aligned >1 times
62.01% overall alignment rate
2000 reads; of these:
  1596 (79.80%) were paired; of these:
    1596 (100.00%) aligned concordantly 0 times
    0 (0.00%) aligned concordantly exactly 1 time
    0 (0.00%) aligned concordantly >1 times
    ----
    1596 pairs aligned concordantly 0 times; of these:
  

6000 reads; of these:
  6000 (100.00%) were unpaired; of these:
    2657 (44.28%) aligned 0 times
    3286 (54.77%) aligned exactly 1 time
    57 (0.95%) aligned >1 times
55.72% overall alignment rate
6000 reads; of these:
  6000 (100.00%) were unpaired; of these:
    1555 (25.92%) aligned 0 times
    4220 (70.33%) aligned exactly 1 time
    225 (3.75%) aligned >1 times
74.08% overall alignment rate
6000 reads; of these:
  6000 (100.00%) were unpaired; of these:
    3751 (62.52%) aligned 0 times
    2236 (37.27%) aligned exactly 1 time
    13 (0.22%) aligned >1 times
37.48% overall alignment rate

In [26]:
%%bash -s "$folder1"
for infile in output-tda/$1/out-bowtie/*.sam
do 
	base=$(basename ${infile} .sam)
	samtools view ${infile} -o output-tda/$1/bam/${base}.bam
    samtools sort ${infile} -o output-tda/$1/sorted/${base}_sorted.sam
done

In [27]:
%%bash -s "$folder1"
for infile in output-tda/$1/bam/*.bam
do 
	base=$(basename ${infile} .bam)
	samtools sort ${infile} -o output-tda/$1/sorted/${base}_sorted.bam
	samtools view ${infile} > output-tda/$1/bam/${base}.txt
done

In [28]:
%%bash -s "$folder1"
#cd ~/GIT/clavibacter/scripts-tda/
ls output-tda/$1/bam/*.txt | while read line
do
    base=$(basename ${line} .txt)
    cut -f 1-6 $line > output-tda/$1/tablas/${base}_extrac.txt
done

In [29]:
ruta_tablas = '/home/shaday/GIT/TDA-Metaganomas/output-tda/'+folder1+'/tablas/'
contenido = list(listdir_nohidden(ruta_tablas))
genomes = []
df=[]
for i in contenido:
    dftt = pd.read_csv(ruta_tablas + i, sep='\t', header=None) #importar las tablas para cada genoma
    dftt['Organims']=i[10:-11] #agregar el nombre del organismo
    df.append(dftt) #agregar al dataframe
    genomes.append(i[16:-11])
    
df=pd.concat(df)
df.columns= ['Qname','Flag', 'Rname', 'Pos','MapQ','CIGAR','Organims'] #cambiar nombre a las columnas del dataframe
df 

,Qname,Flag,Rname,Pos,MapQ,CIGAR,Organims
0,NZ_CP033723.2_0,153,NZ_CP040788.1,2492609,24,150M,tessellarius_ATCC_33566
1,NZ_CP033723.2_5,97,NZ_CP040788.1,1424421,0,150M,tessellarius_ATCC_33566
2,NZ_CP033723.2_5,145,NZ_CP040788.1,1424301,0,150M,tessellarius_ATCC_33566
3,NZ_CP033723.2_6,97,NZ_CP040788.1,1978268,24,150M,tessellarius_ATCC_33566
4,NZ_CP033723.2_6,145,NZ_CP040788.1,1978148,24,150M,tessellarius_ATCC_33566
...,...,...,...,...,...,...,...
2625,NZ_CP086350.1_135,97,NZ_CP086350.1,11284,42,150M,subsp_nebraskensis_419B
2626,NZ_CP086350.1_135,145,NZ_CP086350.1,11164,42,150M,subsp_nebraskensis_419B
2627,NZ_CP040788.1_1,73,NZ_CP086350.1,2766426,0,150M,subsp_nebraskensis_419B
2628,NZ_CP040788.1_3,97,NZ_CP086350.1,1910332,3,150M,subsp_nebraskensis_419B


In [10]:
X=df['Organims'].unique().tolist()
len(X)
dic_Organisms={}
for k in range(len(X)):
    dic_Organisms[X[k]]="X{0}".format(k+1)
dic_Organisms
df2=df.replace({"Organims": dic_Organisms}) 

X2=df2['Organims'].unique().tolist() #lista de genomas 
A=powerset(X2) #conjunto potencia
A=A[1:] #eliminiamos el conjunto vacio
A=sorted(A,key=len)

In [33]:
df2

,Qname,Flag,Rname,Pos,MapQ,CIGAR,Organims
0,NZ_CP033723.2_0,97,NZ_CP021038.1,2187151,40,150M,X1
1,NZ_CP033723.2_0,145,NZ_CP021038.1,2187031,40,150M,X1
2,NZ_CP033723.2_1,97,NZ_CP021038.1,2176151,0,150M,X1
3,NZ_CP033723.2_1,145,NZ_CP021038.1,2176031,0,150M,X1
4,NZ_CP033723.2_2,97,NZ_CP021038.1,2208556,24,150M,X1
...,...,...,...,...,...,...,...
1054,NC_009480.1_217,16,NC_010407.1,333936,40,150M,X4
1055,NC_009480.1_218,0,NC_010407.1,2459844,8,9M2I139M,X4
1056,NC_009480.1_219,16,NC_010407.1,1446996,40,150M,X4
1057,NC_009480.1_220,0,NC_010407.1,424724,23,150M,X4


In [11]:
X_organisms=list(dic_Organisms.values())
X_organisms

['X1', 'X2', 'X3', 'X4']

In [30]:
df.to_csv('CSV/MapeoReads-2TP-6organims.csv')

In [44]:
dic={}
l=0
Reads=df2['Qname'].unique().tolist()
for k in Reads:
    valores=list(df2.loc[df2.Qname==k].iloc[:,6])
    dic[k]=valores


In [34]:

p1=set(list(df2.loc[df2.Organims=='X1'].iloc[:,0]))
p2=set(list(df2.loc[df2.Organims=='X2'].iloc[:,0]))
p3=set(list(df2.loc[df2.Organims=='X3'].iloc[:,0]))
p4=set(list(df2.loc[df2.Organims=='X4'].iloc[:,0]))

In [50]:
list_reads =[set(list(df2.loc[df2.Organims==i].iloc[:,0])) for i in X_organisms]

In [51]:
all_intersections = [set.intersection(*x) for n in range(len(list_reads)) for x in combinations(list_reads, n+1)]

len(all_intersections)
    

15

In [55]:
A

[['X1'],
 ['X2'],
 ['X3'],
 ['X4'],
 ['X1', 'X2'],
 ['X1', 'X3'],
 ['X2', 'X3'],
 ['X1', 'X4'],
 ['X2', 'X4'],
 ['X3', 'X4'],
 ['X1', 'X2', 'X3'],
 ['X1', 'X2', 'X4'],
 ['X1', 'X3', 'X4'],
 ['X2', 'X3', 'X4'],
 ['X1', 'X2', 'X3', 'X4']]

In [54]:
lenIntersecciones(all_intersections)

[848, 590, 829, 681, 557, 711, 650, 549, 522, 630, 522, 508, 602, 495, 484]

In [53]:
set(sorted(lenIntersecciones(all_intersections))

[484, 495, 508, 522, 522, 549, 557, 590, 602, 630, 650, 681, 711, 829, 848]

In [36]:
p=set.intersection(p1,p2,p3,p4)
len(p)

484

In [31]:
# Calculamos SY para cada Y en 2**X
SY=[]
SY= Sy(dic,A)


In [ ]:
#Exportamos Sy a txt
for k in range(len(SY)):
    with open('output-tda/'+folder1+'/Seads/Sy{0}.txt'.format(k), 'w') as f:
        for line in SY[k]:
            f.write(f"{line}\n")

In [ ]:
#añadir un tab al final de cada linea de los reads
%%bash -s "$folder1"
ls output-tda/$1/SyReads/*.txt | while read file
do
perl -p -i -e 's/$/\t/' $file
done

In [ ]:
#extraer los reads de  de cada SY
%%bash -s "$folder1"
for line in output-tda/$1/SyReads/*.txt 
do
    bash 02-extract-reads-for-SY2.sh $1 $line
done

In [ ]:
#Calcular los depth de cada SY
%%bash -s "$folder1"
for line in output-tda/$1/sam_extract/*.bam
do
    base=$(basename ${line} .bam)
    samtools depth ${line} > output-tda/$1/depth/${base}.txt
done

In [11]:
#importar los depth
ruta_depth = '/home/shaday/GIT/clavibacter/scripts-tda/output-tda/'+folder1+'/depth/'
contenido_depth = os.listdir(ruta_depth)
contenido_depth

['Sy3-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy4-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy6-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy5-Simulados-subsp_nebraskensis_61-1_sorted.txt',
 'Sy2-Simulados-subsp_nebraskensis_61-1_sorted.txt',
 'Sy1-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy0-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy6-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy3-Simulados-subsp_nebraskensis_61-1_sorted.txt',
 'Sy2-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy1-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy5-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy2-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy6-Simulados-subsp_nebraskensis_61-1_sorted.txt',
 'Sy4-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy3-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy5-Simulados-subsp_michi_contigs_sorted.txt',
 'Sy0-Simulados-subsp_capsici_1101_sorted.txt',
 'Sy4-Simulados-subsp_nebraskensis_61-1_sorted.txt',
 'Sy0-Simulados-subsp_nebraskensis_61-1_sorted.txt',
 'S

In [39]:
#Calculo de profundidades
dep=[]
for i in range(len(A)):
    globals()["Dep_Y{}".format(i)]=depth_Y2('Sy{0}'.format(i),A[i])
    dep.append(depth_Y2('Sy{0}'.format(i),A[i]))

In [40]:
dp = pd.DataFrame(dep,columns=dic_Organisms.values())
dp

,X1,X2,X3
0,1.355061,0.000000,0.000000
1,0.000000,1.007605,0.000000
2,0.251599,0.251560,0.000000
3,0.000000,0.000000,2.593790
4,0.449539,0.000000,0.440425
5,0.000000,0.258541,0.258459
6,0.118768,0.118800,0.118809


In [18]:
A

[['X1'],
 ['X2'],
 ['X1', 'X2'],
 ['X3'],
 ['X1', 'X3'],
 ['X2', 'X3'],
 ['X1', 'X2', 'X3']]

In [152]:
## Calculamos todos los t, para los diferentes niveles de filtracion
tiempo=[]
for col in dp:
    tiempo.extend(dp[col].unique().tolist())
#tiempo.sort()
tiempo=list(set(tiempo))
tiempo.sort()
tiempo

[0.0,
 0.1187677984990415,
 0.11879989292260427,
 0.11880918748094202,
 0.2515601861645864,
 0.2515989217189069,
 0.25845883139352,
 0.25854054054054054,
 0.4404248929280267,
 0.44953883883350215,
 1.0076051253218121,
 1.3550605516762937,
 2.593790478309586]

In [53]:
# dp['Y']=A
# dp
dp

,X1,X2,X3
0,1.355061,0.000000,0.000000
1,0.000000,1.007605,0.000000
2,0.251599,0.251560,0.000000
3,0.000000,0.000000,2.593790
4,0.449539,0.000000,0.440425
5,0.000000,0.258541,0.258459
6,0.118768,0.118800,0.118809


In [253]:
t=tiempo[1]
t

0.1187677984990415

In [155]:
A

[['X1'],
 ['X2'],
 ['X1', 'X2'],
 ['X3'],
 ['X1', 'X3'],
 ['X2', 'X3'],
 ['X1', 'X2', 'X3']]

In [202]:
#Calculamos 
mat = dp.apply(lambda row: [i if i > t else 0 for i in row], axis=1)
mat

0                       [1.3550605516762937, 0, 0]
1                       [0, 1.0076051253218121, 0]
2      [0.2515989217189069, 0.2515601861645864, 0]
3                        [0, 0, 2.593790478309586]
4     [0.44953883883350215, 0, 0.4404248929280267]
5       [0, 0.25854054054054054, 0.25845883139352]
6    [0, 0.11879989292260427, 0.11880918748094202]
dtype: object

In [144]:
mat = dp.apply(lambda row: np.nonzero([i if i > .36 else 0 for i in row])[0]+1, axis=1)
mat

0       [1]
1       [2]
2        []
3       [3]
4    [1, 3]
5        []
6        []
dtype: object

In [135]:
def distance(t,y):
    mat=dp.apply(lambda row: np.nonzero([i if i > t else 0 for i in row])[0]+1, axis=1)
    return mat[y]
    

In [204]:
distance(t,2)

array([1, 2])

In [ ]:
dic_Yt={}
for k in range(len(A)):
    dic_Yt[k]=dir

In [47]:
A

[['X1'],
 ['X2'],
 ['X3'],
 ['X1', 'X2'],
 ['X1', 'X3'],
 ['X2', 'X3'],
 ['X1', 'X2', 'X3']]

In [251]:
#dp.to_csv('CSV/Depth1TP.csv')


In [ ]:
#dp.loc[dp['Y']==[X1]]
#print(dp.loc[dp['Y'] == 'X2'])
#dp.loc

#dic={}
#l=0
tem=dp['Y'].tolist()
for k in tem:
    valores=list(dp.loc[dp.Y==k])
#    dic[k]=valores
#dp['Y'].tolist()


In [ ]:

    for y in A:
    k=1
    list(dp.loc[dp.Y==y].iloc[:,k])

In [55]:
A
#dp.reindex(A)

[['X1'],
 ['X2'],
 ['X1', 'X2'],
 ['X3'],
 ['X1', 'X3'],
 ['X2', 'X3'],
 ['X1', 'X2', 'X3']]

In [89]:
organisms=list(dic_Organisms.values())

In [102]:
set(organisms[0]).issubset(set(A[2]))


False

In [157]:
powerset([1,2])



[[], [1], [2], [1, 2]]

In [158]:
powerset(list(distance(.2,2)))

[[], [1], [2], [1, 2]]

In [105]:
set(A[2])

{'X1', 'X2'}

In [169]:
d = {}
d[0]=set([])
d[1]=set([1])
d[2]=set([2])
d[3]=set([3])
d[4]=set([1,2])
d[5]=set([1,3])
d[6]=set([2,3])
d[7]=set([1,2,3])

In [170]:
d

{0: set(),
 1: {1},
 2: {2},
 3: {3},
 4: {1, 2},
 5: {1, 3},
 6: {2, 3},
 7: {1, 2, 3}}

In [171]:
list(d.keys())[list(d.values()).index(set([2,3]))]

6

In [172]:
def d_inv(s):
    return list(d.keys())[list(d.values()).index(s)]

In [173]:
d_inv(set([1,2]))

4

In [181]:
def subsets(s):
    return list(map(lambda x: d_inv(set(x)),powerset(d[s])))

In [177]:
list(map(lambda x: d_inv(set(x)),powerset(d[4])))

[0, 1, 2, 4]

In [185]:
cadena = subsets(4)
cadena

[0, 1, 2, 4]

In [189]:
t=.25
res = list(map(lambda y:set(distance(t,y)),cadena ))
res

[{1}, {2}, {1, 2}, {1, 3}]

In [191]:
inter = set.intersection(*res)
inter

set()

In [216]:
type(d[7])

set

In [195]:
import itertools as it
list(it.permutations('123'))

[('1', '2', '3'),
 ('1', '3', '2'),
 ('2', '1', '3'),
 ('2', '3', '1'),
 ('3', '1', '2'),
 ('3', '2', '1')]

In [199]:
list(map("".join, it.permutations('123')))

['123', '132', '213', '231', '312', '321']

In [15]:
# complejo simplicial cech
A

[['X1'],
 ['X2'],
 ['X1', 'X2'],
 ['X3'],
 ['X1', 'X3'],
 ['X2', 'X3'],
 ['X1', 'X2', 'X3']]

In [66]:
lenSy(dic,A)


[1198, 227, 127, 2176, 450, 351, 1471]

In [64]:
A[2]

['X1', 'X2']

In [37]:
my_sets = [p1,p2,p3,p4]

all_intersections = [set.intersection(*x) for n in range(len(my_sets)) for x in combinations(my_sets, n+1)]

len(all_intersections)

15

In [40]:
all_intersections[4] == set.intersection(p1,p2)

True

In [23]:
lenIntersecciones(all_intersections)

[3246, 2176, 4448, 1598, 1921, 1822, 1471]

In [48]:
dic_particion={}
for i in range(len(A)):
    dic_particion[i]=A[i]
dic_particion

{0: ['X1'],
 1: ['X2'],
 2: ['X3'],
 3: ['X1', 'X2'],
 4: ['X1', 'X3'],
 5: ['X2', 'X3'],
 6: ['X1', 'X2', 'X3']}

In [49]:
max(lenIntersecciones(all_intersections))


4448